# Feature Pyramid

![feature pyramid](https://lilianweng.github.io/lil-log/assets/images/featurized-image-pyramid.png)

This is a more advanced approach.
We can stride down and up _within a repeated block_.
Skip connections prevent losing too much information.

## Caveats

As we are decreasing the size, increasing the size _and_ using skip connections this block is very sensitive to the image size.
If the image size is not a multiple of 4 you will end up with rounding errors, breaking your model.
Furthermore you can't go smaller than 1, so you cannot apply only this layer.

These conditions end up enforcing a fixed image size on your network.
This kind of limitation is why, in practice, image processing works over fixed size images.

## Upsampling

Finally, how do we scale up?
We can scale up using an `nn.Upsample` layer.
We use the same kind of scaling that is used in a paint program.
You can use a convolution to scale up (called a deconvolution) however it is not a good idea.

The upsample layer can upscale using a number of different scaling algorithms.
`nearest` is the simplest (just copy the pixels) and `bilinear` is the one you're probably used to when editing images.

In [ ]:
from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim

import numpy as np
from torchvision import datasets, transforms

from tqdm import tqdm

import PIL

In [ ]:
nn.Upsample?

In [ ]:
class SkipSequential(nn.Sequential):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_prime = super().forward(x)
        return torch.cat([x, x_prime], dim=1)

In [ ]:
def pyramid_block(in_channels: int, out_channels: int) -> nn.Module:
    # outer -> middle -> inner -> middle -> outer
    
    # input_size: out_channels
    # output_size: out_channels + out_channels
    inner = SkipSequential(
        nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(2, 2),
            padding=(1, 1),
            padding_mode='reflect',
        ),
        nn.Tanh(),
        nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            padding=(1, 1),
            padding_mode='reflect',
        ),
        nn.Tanh(),
        nn.Upsample(scale_factor=2)
    )
    
    # input_size: in_channels
    # output_size: in_channels + out_channels + out_channels
    middle = SkipSequential(
        nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(2, 2),
            padding=(1, 1),
            padding_mode='reflect',
        ),
        nn.Tanh(),
        inner,
        nn.Upsample(scale_factor=2)
    )
    
    # input_size: in_channels
    # output_size: out_channels
    outer = nn.Sequential(
        middle,
        nn.Conv2d(
            in_channels=in_channels + out_channels + out_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(2, 2),
            padding=(1, 1),
            padding_mode='reflect',
        ),
        nn.Tanh(),
    )
    return outer

---

Because this shrinks and expands, there is a point past which this will not work.

In [ ]:
pyramid_block(3, 16)(torch.zeros(1, 3, 32, 32)).shape

In [ ]:
pyramid_block(3, 16)(torch.zeros(1, 3, 4, 4)).shape